In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
# Define reference state in J, mol, K, Pa
P_ref = 0.01 * 10**6
T_ref = 45 + 273
V_ref = 14.67 * 0.018
U_ref = 2437.2 * 18
H_ref = 2583.9 * 18
S_ref = 8.149 * 18

# Parameters for EOS
Cp = 1.996 * 18
Tc = 647.3
Pc = 22.09 * 10**6
a = 27/64 * 8.314**2 * Tc**2 / Pc
b = 1/8 * 8.314 * Tc / Pc

def predict(P, T):
  # Define starting quantities
  P_cal = P_ref
  T_cal = T_ref
  V_cal = V_ref
  U_cal = U_ref
  H_cal = H_ref
  S_cal = S_ref

  # Convert P to Pascal, and T to Kelvin
  P = P * 10**6
  T = T + 273

  # Integrate with respect to pressure using rectangular method
  dP = 100
  points = np.arange(start=P_cal, stop=P+dP, step=dP)  
  for i in range(len(points)-1):
    P = points[i]
    bottom = (P + a/V_cal**2 - (2*a*(V_cal-b)) / V_cal**3)
    dVdP = -(V_cal - b) / bottom
    dVdT = 8.314 / bottom
    V_next = V_cal + dVdP * dP
    H_next = H_cal + (-T_cal * dVdT + V_cal) * dP
    S_next = S_cal + (-dVdT) * dP

    V_cal = V_next
    H_cal = H_next
    S_cal = S_next

  P_cal = points[-1]
  U_cal = H_cal - (P_cal * V_cal)

  # Integrate with respect to temperature using rectangular method
  dT = 0.1
  points = np.arange(start=T_cal, stop=T+dT, step=dT)
  for i in range(len(points)-1):
    T = points[i]
    bottom = (P_cal + a/V_cal**2 - (2*a*(V_cal-b)) / V_cal**3)
    dVdT = 8.314 / bottom
    V_next = V_cal + dVdT * dT
    H_next = H_cal + Cp * dT
    S_next = S_cal + (Cp / T) * dT

    V_cal = V_next
    H_cal = H_next
    S_cal = S_next
    
  T_cal = points[-1]
  U_cal = H_cal - (P_cal * V_cal)

  # Convert back to kJ, kg, C, MPa
  P_cal = P_cal / 10**6
  T_cal = T_cal - 273
  V_cal = V_cal / 0.018
  U_cal = U_cal / 18
  H_cal = H_cal / 18
  S_cal = S_cal / 18

  return np.array([P_cal, T_cal, V_cal, U_cal, H_cal, S_cal])

# Import data
data = pd.read_csv("drive/My Drive/H2O_Super.csv")
X_train, X_test = train_test_split(data.values, train_size=0.8, random_state=1)

# Calculate predictions
X_predict = np.zeros((np.size(X_test, 0), np.size(X_test, 1)))
for i in range(np.size(X_test, 0)):
  X_predict[i, :] = predict(X_test[i, 0], X_test[i, 1])

In [14]:
# Compute normalized SSE
X_mean = np.mean(X_train, 0)
X_std = np.std(X_train, 0)

X_test = (X_test - X_mean) / X_std
X_predict = (X_predict - X_mean) / X_std

error = X_test - X_predict
ind_err = np.sum(error ** 2, axis=0)[2:]
print('Volume SSE: {:.4f}'.format(ind_err[0]))
print('Energy SSE: {:.4f}'.format(ind_err[1]))
print('Enthalpy SSE: {:.4f}'.format(ind_err[2]))
print('Entropy SSE: {:.4f}'.format(ind_err[3]))
SSE = np.sum(error ** 2)
print('Total SSE: {:.4f}'.format(SSE))

Volume SSE: 0.0014
Energy SSE: 243.0763
Enthalpy SSE: 69.0767
Entropy SSE: 55.9281
Total SSE: 368.0825
